# Enviroment

In [1]:
!pip install pymupdf nltk chromadb sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 20.0 MB/s eta 0:

In [2]:
# ========================================
# 标准库导入
# ========================================
import os
import re
import sys
import time
import datetime
import hashlib
from pathlib import Path
from threading import Thread, Lock
from typing import List, Dict, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed

# ========================================
# 文件/文档处理
# ========================================
import fitz  # PyMuPDF

# ========================================
# NLP/ML相关库
# ========================================
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import jieba
import numpy as np
import torch
from torch import cuda
# from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# ========================================
# 向量数据库/存储
# ========================================
import chromadb
from chromadb.utils import embedding_functions

# ========================================
# 接口/可视化
# ========================================
# import gradio as gr
from tqdm import tqdm

# ========================================
# 第三方服务/API
# ========================================
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_exponential

# ========================================
# 调试/工具
# ========================================
import traceback

# ========================================
# 已注释的备用依赖
# ========================================
# from elasticsearch import Elasticsearch

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Config

In [29]:
# =============================
# 配置参数（统一管理）
# =============================
class Config:
    # 多线程配置
    NUM_WORKERS = 8
    USE_MULTITHREAD = True

    # 路径配置
    UPLOAD_DIR = "/content/drive/MyDrive/513"
    DB_PATH = "/content/drive/MyDrive/513/chroma_db"

    # 文件配置
    SUPPORTED_EXTS = [".pdf", ".PDF"]  # 支持的文件扩展名
    MAX_FILE_SIZE = 100 * 1024 * 1024  # 100MB文件大小限制

    # 处理参数
    CHUNK_SIZE = 300
    OVERLAP = 50
    TOP_K = 100        # 初始检索数量
    TOP_K_FINAL = 5   # 最终返回数量
    BATCH_SIZE = 196

    # 模型配置
    EMBEDDING_MODEL = "BAAI/bge-m3"
    RERANKER_MODEL = "BAAI/bge-reranker-large"
    API_KEY = "c8bc2c37-95f2-4202-b12d-f9392185408f"
    BASE_URL = "https://ark.cn-beijing.volces.com/api/v3"
    MAX_SEQ_LENGTH = 512

    # 数据库配置
    COLLECTION_NAME = "HuShen300"
    INSERT_BATCH = 1000

    MODELS = {
      "DeepSeek_R1": "deepseek-r1-250120",
      "DeepSeek_V3": "deepseek-v3-250324"
    }
    MAX_TOKENS = 3000
    TEMPERATURE=0.7


# Vector DB Initializaition

In [6]:
client = chromadb.PersistentClient(path="/content/drive/MyDrive/513/chroma_db/")

In [7]:
collections = client.list_collections()

print("已有集合：", [col.name for col in collections])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

已有集合： ['DocumentDB', 'HuShen300']


In [8]:
model = SentenceTransformer(Config.EMBEDDING_MODEL)
model.max_seq_length = Config.MAX_SEQ_LENGTH

In [9]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=Config.EMBEDDING_MODEL,
            device="cuda",
            normalize_embeddings=True
        )

collection = client.get_or_create_collection(
            name="HuShen300",
            embedding_function=embedding_func,
            metadata={"hnsw:space": "cosine"},
        )

In [15]:
# %% ##########################
# 增强版PDF处理模块（支持错误隔离）
###############################
import logging
from typing import Dict, Tuple
from pathlib import Path
import fitz
import re

# 配置日志
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

class PDFProcessor:
    @staticmethod
    def pdf_to_text(file_path: str) -> Tuple[str, bool]:
        """
        安全转换PDF为文本
        返回: (提取文本, 是否成功)
        """
        try:
            text = []
            # 启用容错模式打开文件
            with fitz.open(file_path) as doc:
                for page in doc:
                    text.append(page.get_text())
            return "\n".join(text), True

        except fitz.FileDataError as e:
            logger.error(f"文件结构损坏跳过: {file_path} | 错误: {str(e)}")
            return "", False

        except Exception as e:
            logger.error(f"未知错误跳过文件: {file_path} | 错误类型: {type(e).__name__}")
            return "", False

    @staticmethod
    def clean_text(text: str) -> str:
        """优化版文本清洗"""
        # 保留段落分隔
        text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)  # 替换单换行为空格
        text = re.sub(r'\n{2,}', '\n', text)         # 合并多个换行
        text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)  # 移除控制字符
        return text.strip()

    @staticmethod
    def batch_process_pdfs(folder_path: str) -> Tuple[Dict[str, str], Dict[str, str]]:
        """
        批量处理PDF文件
        返回: (成功文件字典, 失败文件字典)
        """
        success_files = {}
        failed_files = {}
        folder_path = Path(folder_path)

        if not folder_path.is_dir():
            raise ValueError(f"无效的文件夹路径: {folder_path}")

        # 获取所有PDF文件（按修改时间排序）
        pdf_files = sorted(
            folder_path.glob("**/*.pdf"),
            key=lambda x: x.stat().st_mtime,
            reverse=True
        )

        for pdf_file in pdf_files:
            file_path = str(pdf_file)
            try:
                # 带重试机制的文本提取
                text, status = PDFProcessor.pdf_to_text(file_path)
                if not status or not text:
                    failed_files[pdf_file.name] = "内容提取失败"
                    continue

                cleaned_text = PDFProcessor.clean_text(text)
                if not cleaned_text:
                    failed_files[pdf_file.name] = "清洗后内容为空"
                    continue

                success_files[pdf_file.name] = cleaned_text
                logger.info(f"成功处理: {pdf_file.name} ({len(text)}字符)")

            except KeyboardInterrupt:
                logger.warning("用户中断处理流程")
                break

            except Exception as e:
                logger.exception(f"未捕获异常: {pdf_file.name}")
                failed_files[pdf_file.name] = str(e)
                continue

        return success_files, failed_files

In [11]:
# %% ##########################
# 文本分块模块
###############################
class TextChunker:

    @staticmethod
    def split_into_chunks(text: str, chunk_size: int) -> List[str]:
      """基于句子的分割"""
      sentences = sent_tokenize(text)
      chunks = []
      current_chunk = []
      current_length = 0

      for sent in sentences:
          sent_len = len(sent)

          # 处理超长单个句子
          if sent_len > chunk_size:
              if current_chunk:  # 先保存已有块
                  chunks.append(" ".join(current_chunk))
                  current_chunk = []
              # 强制分割长句
              chunks.extend([sent[i:i+chunk_size] for i in range(0, sent_len, chunk_size)])
              current_length = 0
              continue

          # 正常累加逻辑
          if current_length + sent_len > chunk_size:
              chunks.append(" ".join(current_chunk))
              # 智能重叠（保留完整句子）
              overlap = []
              overlap_len = 0
              while current_chunk and (overlap_len + len(current_chunk[-1])) <= chunk_size * 0.2:
                  overlap.insert(0, current_chunk.pop())
                  overlap_len += len(overlap[0])
              current_chunk = overlap
              current_length = overlap_len

          current_chunk.append(sent)
          current_length += sent_len

      if current_chunk:
          chunks.append(" ".join(current_chunk))

      return chunks


    # 在TextChunker类中可添加自适应分块算法

    @staticmethod
    def dynamic_chunking(text: str, min_size: int = 200, max_size: int = 500) -> List[str]:
        """
        优化的动态分块算法(支持中英文金融文档)
        Args:
            text: 输入文本(需预先清洗)
            min_size: 最小分块长度(建议200-300)
            max_size: 最大分块长度(建议500-800)
        Returns:
            List[str]: 符合长度限制的语义块列表
        """
        if not text:
            return []

        # 优先级排序的断点符号(分数，符号长度)
        BREAKERS = {
            '\n\n': (0.95, 2),   # 段落分隔
            '。': (0.85, 1),     # 句子结束
            '!': (0.8, 1), '?': (0.8, 1),  # 感叹/疑问
            '；': (0.7, 1), ';': (0.7, 1), # 分号
            '，': (0.5, 1), ',': (0.5, 1) # 逗号
        }

        chunks = []
        start = 0
        text_len = len(text)

        while start < text_len:
            # 计算当前窗口边界
            end = min(start + max_size, text_len)
            window = text[start:end]

            best_split = -1
            best_score = 0

            # 逆向扫描寻找最优断点
            for pos in range(len(window)-1, min_size-1, -1):
                # 获取当前位置上下文
                curr_char = window[pos]
                prev_char = window[pos-1] if pos > 0 else ''
                next_char = window[pos+1] if pos < len(window)-1 else ''

                # 评估所有可能的断点
                for br, (score, br_len) in BREAKERS.items():
                    # 匹配多字符断点
                    if br_len == 2 and pos >= 1:
                        if window[pos-1:pos+1] == br:
                            current_score = score
                            split_pos = pos + 1  # 包含断点符号
                            break
                    elif br_len == 1 and curr_char == br:
                        # 处理数字中的逗号(如1,000)
                        if br in [',', '，'] and next_char.isdigit():
                            continue
                        current_score = score
                        split_pos = pos + 1  # 包含断点符号
                        break
                else:
                    continue  # 未找到断点符号

                # 更新最佳分割点
                if current_score > best_score:
                    best_score = current_score
                    best_split = start + split_pos
                    if best_score >= 0.8:  # 高优先级断点立即确定
                        break

            # 确定最终分割位置
            if best_split > start + min_size:
                chunks.append(text[start:best_split].strip())
                start = best_split
            else:
                # 处理剩余文本不足min_size的情况
                if chunks and (len(window) < min_size):
                    chunks[-1] += window  # 合并到前一个块
                else:
                    chunks.append(window.strip())
                start = end

            # 动态重叠处理(不超过max_size的20%)
            overlap = min(int(max_size * 0.2), start - chunks[-1].rfind('\n\n') if '\n\n' in chunks[-1] else 50)
            start = max(start - overlap, 0)

        return [chunk for chunk in chunks if chunk]

In [16]:
pdf_texter = PDFProcessor()
chunker = TextChunker()

In [18]:
#向量插入
@retry(stop=stop_after_attempt(3),
      wait=wait_exponential(multiplier=1, min=2, max=10))
def _insert_batch( batch: dict):
    try:
      collection.add(**batch)
      return True
    except Exception as e:
      print(f"插入重试中... 错误：{str(e)}")
      raise

In [19]:
def pdf_processor( pdf_path: str):
        # 步骤1：PDF转文本
        print("正在解析PDF...")
        raw_text = pdf_texter.pdf_to_text(pdf_path)
        if not raw_text[1]:
          print("PDF解析失败，请检查文件格式")
          return

        # 步骤2：文本清洗和分块
        cleaned_text = pdf_texter.clean_text(raw_text[0])
        chunks = chunker.split_into_chunks(cleaned_text, Config.CHUNK_SIZE)
        chunks = [c for c in chunks if len(c.strip()) > 0]  # 过滤空块
        print(f"生成有效文本块：{len(chunks)} 个")

        # 步骤3：生成向量（可并行优化点）
        print("生成嵌入向量...")
        embeddings = []
        metadata = []
        doc_id = Path(pdf_path).stem  # 获取文件名（不含扩展名）


        #多线程版本
        print("多线程版本\n")
        # 修改后的向量生成部分
        def process_batch(batch, start_idx, doc_id):
            """单批次处理函数（线程安全）"""
            # 生成向量
            vectors = model.encode(
                batch,
                convert_to_tensor=True,
                normalize_embeddings=True,
                show_progress_bar=False
            ).cpu().numpy()

            # 构建元数据
            batch_meta = [{
                "doc_id": doc_id,
                "chunk_index": start_idx + idx,
                "text_snippet": chunk[:100] + "...",
            } for idx, chunk in enumerate(batch)]

            return vectors, batch_meta

        with ThreadPoolExecutor(max_workers=Config.NUM_WORKERS) as executor:
          # 1. 准备任务队列
          futures = []
          for i in range(0, len(chunks), Config.BATCH_SIZE):
              batch = chunks[i:i+Config.BATCH_SIZE]
              # 提交任务（立即非阻塞）
              future = executor.submit(
                  process_batch,
                  batch=batch,
                  start_idx=i,
                  doc_id=doc_id
              )
              futures.append(future)

          # 2. 并行处理 + 进度跟踪
          for future in tqdm(as_completed(futures),
                            total=len(futures),
                            desc="多线程生成向量"):
              batch_vectors, batch_meta = future.result()
              embeddings.extend(batch_vectors)
              metadata.extend(batch_meta)

        # 步骤4：存储到数据库
        print("\n正在写入数据库...")

        with ThreadPoolExecutor(max_workers=Config.NUM_WORKERS) as insert_executor:
          """并行化批量插入"""
          assert len(chunks) == len(embeddings) == len(metadata)

          # 准备所有批次数据（避免在并行中修改）
          batch_list = []
          ids = [f"{m['doc_id']}_{m['chunk_index']}" for m in metadata]
          for i in range(0, len(ids), Config.INSERT_BATCH):
              batch_list.append({
                  "ids": ids[i:i+Config.INSERT_BATCH],
                  "embeddings": embeddings[i:i+Config.INSERT_BATCH],
                  "documents": chunks[i:i+Config.INSERT_BATCH],
                  "metadatas": metadata[i:i+Config.INSERT_BATCH]
              })

          # 并行提交任务
          futures = []
          for batch in batch_list:
              futures.append(insert_executor.submit(_insert_batch, batch))

          # 监控进度与错误
          success_count = 0
          for future in futures:
              try:
                  if future.result():
                      success_count += 1
                  print(f"成功: {success_count}/{len(batch_list)}")
              except Exception as e:
                  print(f"\n永久插入失败: {str(e)}")

          print(f"插入完成 - 成功批次: {success_count}/{len(batch_list)}")


In [20]:
pdf_processor("/content/drive/MyDrive/513/Maotai_2024.pdf")

正在解析PDF...
生成有效文本块：637 个
生成嵌入向量...
多线程版本



多线程生成向量: 100%|██████████| 4/4 [00:05<00:00,  1.34s/it]



正在写入数据库...
成功: 1/2
成功: 2/2
插入完成 - 成功批次: 2/2


In [ ]:
#向量插入
@retry(stop=stop_after_attempt(3),
      wait=wait_exponential(multiplier=1, min=2, max=10))
def _insert_batch( batch: dict):
    try:
      collection.add(**batch)
      return True
    except Exception as e:
      print(f"插入重试中... 错误：{str(e)}")
      raise

In [ ]:
pdf_processor("/content/drive/MyDrive/513/Maotai_2024.pdf")

正在解析PDF...
生成有效文本块：565 个
生成嵌入向量...
多线程版本



多线程生成向量: 100%|██████████| 12/12 [00:20<00:00,  1.74s/it]



正在写入数据库...
成功: 1/2
成功: 2/2
插入完成 - 成功批次: 2/2


In [23]:
!pip install pandas openpyxl akshare

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 113.5 MB/s eta 0:00:00
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5615 sha256=a2a4dcd667655cba589046cb3cc909652b5660719c6239e4be20782f7aa4c2ba
  Stored in directory: /root/.cache/pip/wheels/7d/b8/16/48bbd715040679311fa68cb564ad24a97d9a67fea5d4c848c8
Successfully built jsonpath


In [24]:
import akshare as ak
import pandas as pd
from tqdm import tqdm  # 进度条支持

import os
import requests
import threading
from queue import Queue
from urllib.parse import urlparse
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

class ThreadSafeCounter:
    """线程安全计数器"""
    def __init__(self):
        self._value = 0
        self._lock = threading.Lock()

    def increment(self):
        with self._lock:
            self._value += 1

    @property
    def value(self):
        with self._lock:
            return self._value

def download_single_pdf(url, target_folder, error_queue, counter, txt_name):
    """单个PDF下载线程（新增txt_name参数）"""
    try:
        path = urlparse(url).path
        path_parts = path.split('/')
        if len(path_parts) < 4:
            raise ValueError("URL路径格式错误")

        date_str = path_parts[-2]
        base_name = os.path.splitext(path_parts[-1])[0]

        # 添加TXT文件名前缀
        filename = f"{txt_name}_{date_str}-{base_name}.pdf"  # 新增文件名部分
        save_path = os.path.join(target_folder, filename)

        if os.path.exists(save_path):
            return

        with requests.get(url, stream=True, timeout=(6.1, 30)) as r:
            r.raise_for_status()
            with open(save_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

        counter.increment()

    except Exception as e:
        error_queue.put(f"{url} | 错误：{type(e).__name__}-{str(e)}")

def download_pdfs_from_txts(source_folder: str, target_folder: str, max_workers=20):
    """
    多线程下载PDF主函数
    :param source_folder: TXT文件目录
    :param target_folder: PDF保存目录
    :param max_workers: 并发线程数（建议10-30）
    """
    # 初始化线程安全组件
    error_queue = Queue()
    counter = ThreadSafeCounter()

    # 创建目标目录
    os.makedirs(target_folder, exist_ok=True)

    # 收集所有下载链接
    # 修改URL收集逻辑
    all_urls = []
    for txt_file in os.listdir(source_folder):
        if not txt_file.endswith('.txt') or txt_file.startswith('~$'):
            continue

        # 获取TXT文件名（不含扩展名）
        txt_name = os.path.splitext(txt_file)[0]

        with open(os.path.join(source_folder, txt_file), 'r', encoding='utf-8') as f:
            urls = list({line.strip() for line in f if line.strip()})
            # 存储元组（URL，TXT文件名）
            all_urls.extend( [(url, txt_name) for url in urls] )  # 修改这里

    # 修改任务提交部分
    with tqdm(total=len(all_urls), desc="总进度") as pbar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {
                executor.submit(
                    download_single_pdf,
                    url,           # URL参数
                    target_folder,
                    error_queue,
                    counter,
                    txt_name       # 新增TXT文件名参数
                ): (url, txt_name) for (url, txt_name) in all_urls  # 遍历元组
            }

            for _ in as_completed(futures):
                pbar.update(1)

    # 输出统计信息
    print(f"\n下载完成！成功下载 {counter.value}/{len(all_urls)} 个文件")
    print(f"保存目录：{os.path.abspath(target_folder)}")

    # 输出错误日志
    if not error_queue.empty():
        print("\n错误日志（最多显示20条）：")
        for _ in range(min(20, error_queue.qsize())):
            print(f"• {error_queue.get()}")


In [26]:
input_folder = "/content/drive/MyDrive/513/HuShen300_pdf_download_links/"
output_folder = "/content/HuShen300_pdfs/"

download_pdfs_from_txts(input_folder, output_folder)

总进度: 100%|██████████| 1814/1814 [04:04<00:00,  7.41it/s]


下载完成！成功下载 1808/1814 个文件
保存目录：/content/HuShen300_pdfs

错误日志（最多显示20条）：
• http://static.cninfo.com.cn/finalpage/2025-03-20/1222847583.PDF | 错误：HTTPError-404 Client Error: Not Found for url: http://static.cninfo.com.cn/finalpage/2025-03-20/1222847583.PDF
• http://static.cninfo.com.cn/finalpage/2025-03-27/1222913141.PDF | 错误：HTTPError-404 Client Error: Not Found for url: http://static.cninfo.com.cn/finalpage/2025-03-27/1222913141.PDF
• http://static.cninfo.com.cn/finalpage/2025-04-28/1223350383.PDF | 错误：HTTPError-404 Client Error: Not Found for url: http://static.cninfo.com.cn/finalpage/2025-04-28/1223350383.PDF
• http://static.cninfo.com.cn/finalpage/2025-04-01/1222975145.PDF | 错误：HTTPError-404 Client Error: Not Found for url: http://static.cninfo.com.cn/finalpage/2025-04-01/1222975145.PDF
• http://static.cninfo.com.cn/finalpage/2025-03-28/1223403703.PDF | 错误：HTTPError-404 Client Error: Not Found for url: http://static.cninfo.com.cn/finalpage/2025-03-28/1223403703.PDF
• http://static.cnin

In [ ]:
def process_pdfs(folder_path: str):

    files = [f for f in os.listdir(folder_path)]

    for file in tqdm(files, "向量化数据库"):
            try:
                # 构建完整路径
                file_path = os.path.join(folder_path, file)
                pdf_processor(file_path)
            except Exception as e:
                print(f"处理失败：{file} | 错误：{str(e)}")

In [27]:
import fnmatch

In [28]:
def process_pdfs(folder_path: str, start_from: int = 0, file_pattern: str = "*.pdf"):
    """
    增强版PDF批量处理函数

    :param folder_path: PDF文件夹路径
    :param start_from: 起始文件序号（从0开始计数）
    :param file_pattern: 文件匹配模式，默认处理所有PDF
    """
    # 获取排序后的文件列表
    sorted_files = sorted([
        f for f in os.listdir(folder_path)
        if fnmatch.fnmatch(f, file_pattern)
    ], key=lambda x: x.lower())  # 不区分大小写排序

    # 有效性检查
    if start_from >= len(sorted_files):
        print(f"警告：起始位置 {start_from} 超过文件总数 {len(sorted_files)}")
        return

    # 创建带序号显示的进度条
    with tqdm(
        sorted_files[start_from:],
        desc=f"处理文件 [{start_from}-{len(sorted_files)-1}]",
        bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]"
    ) as pbar:
        for idx, file in enumerate(pbar, start=start_from):
            try:
                file_path = os.path.join(folder_path, file)
                pbar.set_postfix(file=f"{idx+1}.{file[:10]}...")  # 显示当前序号
                pdf_processor(file_path)
            except Exception as e:
                print(f"\n失败文件：{file} | 错误：{str(e)}")
                # 自动保存断点位置
                with open("last_position.txt", "w") as f:
                    f.write(str(idx))
                if input("是否继续？(y/n)").lower() != 'y':
                    break

In [ ]:
process_pdfs("/content/HuShen300_pdfs/")

处理文件 [0-1807]:   0%|          | 0/1808 [00:00<?]

正在解析PDF...
生成有效文本块：969 个
生成嵌入向量...
多线程版本




多线程生成向量: 100%|██████████| 5/5 [00:07<00:00,  1.48s/it]



正在写入数据库...


处理文件 [0-1807]:   0%|          | 1/1808 [00:35<17:35:24]

...


多线程生成向量: 100%|██████████| 9/9 [00:11<00:00,  1.26s/it]



正在写入数据库...


处理文件 [0-1807]:  42%|████▏     | 753/1808 [5:11:21<9:39:12]

成功: 1/2
成功: 2/2
插入完成 - 成功批次: 2/2
正在解析PDF...
生成有效文本块：1698 个
生成嵌入向量...
多线程版本




多线程生成向量:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
from google.colab import runtime
runtime.unassign()